# Option Valuation

### BUSI 722: Data-Driven Finance II
### Kerry Back, Rice University

### Comparison pricing

- How do you decide if a house is fairly priced?
- An analogue to price / square foot for valuing companies is price-to-earnings.
- For valuing an option, we could use other options or we can just start with the underlying price.
- Option value = intrinsic value + adjustment for time and uncertainty.

### Replication

- If we could create the option value at maturity by dynamically trading the underlying, then the value of the option should be the cost of the underlying portfolio.
- Call $\sim$ long underlying with leverage, so value of call = cost of underlying minus amount borrowed
- Put $\sim$ short underlying not fully collateralized, so value of put = implicit collateral

### Call Option

- Consider a call option with a strike of 105.  
- It ends with a value of 5 if the stock goes to 110 and a value of 0 if the stock goes to 90.
- We want to find its value at the beginning.

![](images/binomial_call_oneperiod.jpg)

### Delta

- Define $\Delta$ as the difference in the option values divided by the difference in the stock values.
- This is $(5-0)/(110-90) = 1/4$.  
- Here is the value of 1/4 share of the stock.
 
![](images/binomial_deltashares_oneperiod.jpg)

### Debt

- Consider borrowing the PV of the bottom value from the previous figure.
- Suppose the interest rate is 5%.  The PV of 22.50 is 21.43. Here is how the debt evolves.

![](images/binomial_liability_oneperiod.jpg)

### Buy $\Delta$ shares on margin

The value of delta shares less the value of the debt is:

![](images/binomial_call_complete_oneperiod.jpg)

### Conclusion

- In this simple example, we can get the call option payoff at maturity by investing 3.57, borrowing 21.43, and buying 1/4 share.
- The value of the call must be 3.57.

## Risk-neutral probability

- If there were no risk premium, the call value would be the expected value discounted at the risk-free rate:

$$C = \frac{p \times \text{\$}5 + (1-p)\times \text{\$}0}{1.05}$$

- where $p=$ up probability.  The stock price would also be the discounted expected value:

$$\text{\$}100 = \frac{p \times \text{\$}110 + (1-p) \times \text{\$}90}{1.05} \quad \Leftrightarrow \quad  S = \frac{p(1+r_u)S + (1-p)(1+r_d)S}{1+r_f} $$

- Solve the stock equation for $p$:

$$p = \frac{r_f - r_d}{r_u - r_d} = \frac{0.05 - (-0.1)}{0.1 - (-0.1)} = \frac{.15}{.2} = 0.75$$

- Substitute into the call option equation.  Obtain

$$C = \frac{0.75 \times \text{\$}5 + 0.25\times \text{\$}0}{1.05} = \text{\$}3.57$$

- Why does this work? Delta hedge argument didn't depend on risk preferences, so we can act as if investors don't require risk premia.

## Risk-neutral probability in two-period example

### Stock dynamics

- A $100 stock goes up by 5% or down by (1/1.05-1) = -4.76% in each of two periods.
- Interest rate is 3% per period

![](images/binomial_two_period.jpg)

### Risk-neutral probability

The risk-neutral probability of "up" is

$$ p = \frac{r_f - r_d}{r_u - r_d} = \frac{0.03 - (-0.0476)}{0.05 - (-0.0476)} = 0.795$$

### Call option with strike = 105

The call evolves as

![](images/call_two_period_1.jpg)

Discounting the expected call value (using prob of up = 0.795) at the risk-free rate yields

![](images/call_two_period_2.jpg)

### Exercise

Price a call with a strike of 95.

## Calibration

- Estimate $\sigma=$ std dev of annual stock return
- Find $r_f =$ annualized continuously compounded interest rate = log(1+annual rate)
- $T=$ time to maturity of an option in years
- $N=$ number of periods in a binomial model

- $\Delta t = T/N=$ period length
- Set the up rate of return as $u = e^{\sigma\sqrt{\Delta t}}-1$ and set $d=1/(1+u)-1$ as in the two-period example
- Set the 1-period interest rate as $r=e^{r_f \Delta t}-1$
- The risk-neutral probability of "up" is

$$p = \frac{r-d}{u-d} = \frac{e^{r_f \Delta t} - e^{-\sigma \Delta t}}{e^{\sigma \Delta t} - e^{-\sigma \Delta t}}$$

### Exercise

Price a six-month call option using a two-period model.

- $\sigma = 0.4$
- $r_f = 0.05$
- $T = 0.5$
- $N = 2$
- $S = 100$
- $K = 95$

## Black-Scholes formulas

- As $N \rightarrow \infty$ the distribution of the stock price at date $T$ converges to lognormal, meaning that the log stock price has a normal distribution.
- The values of European options converge to the Black-Scholes formulas.
- More about American options (and dividends) coming.

### Black-Scholes call formula

In [13]:
import numpy as np 
from scipy.stats import norm 

def BS_call(S, K, T, sigma, r, q=0):
    d1 = np.log(S/K) + (r-q+0.5*sigma**2)*T
    d1 /= sigma*np.sqrt(T)
    d2 = d1 - sigma*np.sqrt(T)
    N1 = norm.cdf(d1)
    N2 = norm.cdf(d2)
    return np.exp(-q*T)*S*N1 - np.exp(-r*T)*K*N2

### Black-Scholes put formula

In [14]:
def BS_put(S, K, T, sigma, r, q=0):
    d1 = np.log(S/K) + (r-q+0.5*sigma**2)*T
    d1 /= sigma*np.sqrt(T)
    d2 = d1 - sigma*np.sqrt(T)
    N1 = norm.cdf(-d1)
    N2 = norm.cdf(-d2)
    return np.exp(-r*T)*K*N2 - np.exp(-q*T)*S*N1

### Example

In [15]:
S, K, T, sigma, r = 100, 95, 0.5, 0.4, 0.05
print(f"Value of call option is ${BS_call(S, K, T, sigma, r):.2f}")
print(f"Value of put option is ${BS_put(S, K, T, sigma, r):.2f}")

Value of call option is $14.89
Value of put option is $7.55


## Dividends and early exercise

### Dividends

- To use binomial model, build a tree for "stock minus PV of future dividends," the future being until the option maturity.
- Try to set tree nodes near ex-dividend dates
- Everything else as before.  As time passes, dividends get paid and "stock minus PV of future dividends" becomes "stock."

### Early exercise

- It may be optimal to exercise an American put at any time (though just after ex-dividend date is better than just before).
- It may be optimal to exercise an American call just before a dividend is paid.
- To use binomial model, replace "discounted risk-neutral expectation of option value" with max of discounted risk-neutral expectation and intrinsic value.